In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go

from unc.utils import load_info
from unc.agents import Agent, DQNAgent
from definitions import ROOT_DIR

In [ ]:
results_fname = Path(ROOT_DIR, "results", "lobster_data.npy")
loaded = load_info(results_fname)

In [ ]:
obs_res = loaded['2']
unc_res = loaded['2o']

In [ ]:
# reduced_unc_obses = get_distilled_obs(unc_res['obs'][0])
unc_obs = unc_res['obs']
at_zero_unc = unc_obs[:, :, 0] == 1

r1_ot_unc = unc_obs[:, :, 3]
r2_ot_unc = unc_obs[:, :, 6]
print(f"r1_ot min: {r1_ot_unc[at_zero_unc].min()}, r1_ot max: {r1_ot_unc[at_zero_unc].max()}")
print(f"r2_ot min: {r2_ot_unc[at_zero_unc].min()}, r2_ot max: {r2_ot_unc[at_zero_unc].max()}")

obs_obs = obs_res['obs']
at_zero_obs = obs_obs[:, :, 0] == 1

r1_ot_obs = obs_obs[:, :, 3]
r2_ot_obs = obs_obs[:, :, 6]
print(f"r1_ot_unc min: {r1_ot_obs[at_zero_obs].min()}, r1_ot_unc max: {r1_ot_obs[at_zero_obs].max()}")
print(f"r2_ot_unc min: {r2_ot_obs[at_zero_obs].min()}, r2_ot_unc max: {r2_ot_obs[at_zero_obs].max()}")

In [ ]:


def get_distilled_obs(traj):
    # first we get position
    pos_one_hot = traj[:, :3]
    pos_traj = np.nonzero(pos_one_hot)[-1]
    ot1 = traj[:, 3]
    ot2 = traj[:, 6]
    reduced_obses = np.stack([pos_traj, ot1, ot2], axis=-1)
    return reduced_obses

In [ ]:
# reduced_unc_obses = get_distilled_obs(unc_res['obs'][0])
traj_obs = obs_res['obs'][0]

reduced_obs_obs = get_distilled_obs(traj_obs)


In [ ]:

fig = plt.figure(figsize=(8, 6), dpi=80)
colors = (z/x.shape[0])[None, :]


ax = fig.add_subplot(projection='3d')
for i in range(1):
    x = r1_ot_obs[i]
    y = r2_ot_obs[i]
    z = np.arange(x.shape[0])
#     x = reduced_obs_obs[:, 1]
#     y = reduced_obs_obs[:, 2]
#     z = reduced_obs_obs[:, 0]

    ax.scatter(x, y, z, c=z, cmap='viridis')
    ax.plot3D(x, y, z, color="black", linewidth=0.5)


ax.set_xlabel("R1 obs")
ax.set_ylabel("R2 obs")
ax.set_zlabel(f"time step")

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', title='action')
ax.set_title("Q-values of Lobster environment at node 0 (uncertainty obs)")

In [ ]:
traj_unc = unc_res['obs'][0]

reduced_obs_unc = get_distilled_obs(traj_unc)

In [ ]:

fig = plt.figure(figsize=(8, 6), dpi=80)
colors = (z/x.shape[0])[None, :]


ax = fig.add_subplot(projection='3d')
for i in range(1):
    x = r1_ot_unc[i]
    y = r2_ot_unc[i]
    z = np.arange(x.shape[0])
#     x = reduced_obs_unc[:, 1]
#     y = reduced_obs_unc[:, 2]
#     z = reduced_obs_unc[:, 0]

    ax.scatter(x, y, z, c=z, cmap='viridis')
    ax.plot3D(x, y, z, color="black", linewidth=0.5)


ax.set_xlabel("R1 obs")
ax.set_ylabel("R2 obs")
ax.set_zlabel(f"time step")

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', title='action')
ax.set_title("Q-values of Lobster environment at node 0 (uncertainty obs)")

In [ ]:
# Here we get all possible states for 2g, at node 0
zero_obs_2g = np.zeros((4, 12))
zero_obs_2g[np.arange(4), [0, 3, 6, 9]] = 1

# so for unc encoding, 1 == observable and collected.
ot_obs_2g = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])

In [ ]:

# here we get all possible observations at node 0 for 2
zero_obs_2 = np.array([[1., 0., 0., 0., 0., 1., 0., 0., 1.]])

# all possible observations at node 0 for 2o
discount = 0.95

obs_2o_range_single = discount ** (np.arange(300) + 1)
obs_2o_range_x, obs_2o_range_y = np.meshgrid(obs_2o_range_single, obs_2o_range_single)
obs_2o_range = np.stack((obs_2o_range_x, obs_2o_range_y), axis=-1)
ot_obs_2o = obs_2o_range.reshape(-1, 2)

zero_obs_2o = np.repeat(zero_obs_2, ot_obs_2o.shape[0], axis=0)
zero_obs_2o[:, [3, 6]] = ot_obs_2o

In [ ]:
fa = 'linear'
obs_agent_fname = Path(ROOT_DIR, 'results', f'2_{fa}_agent.pth')
unc_agent_fname = Path(ROOT_DIR, 'results', f'2o_{fa}_agent.pth')
gt_agent_fname = Path(ROOT_DIR, 'results', f'2g_{fa}_agent.pth')


obs_agent = DQNAgent.load(obs_agent_fname, DQNAgent)
unc_agent = DQNAgent.load(unc_agent_fname, DQNAgent)
gt_agent = DQNAgent.load(gt_agent_fname, DQNAgent)

In [ ]:
all_zero_2_qs = obs_agent.Qs(zero_obs_2, obs_agent.network_params)
all_zero_2o_qs = unc_agent.Qs(zero_obs_2o, unc_agent.network_params)
all_zero_2g_qs = gt_agent.Qs(zero_obs_2g, gt_agent.network_params)

range_2o = all_zero_2o_qs.max() - all_zero_2o_qs.min()
range_2g = all_zero_2g_qs.max() - all_zero_2g_qs.min()


normalized_2o_qs = (all_zero_2o_qs - all_zero_2o_qs.min()) / range_2o
normalized_2g_qs = (all_zero_2g_qs - all_zero_2g_qs.min()) / range_2g

In [ ]:
actions_to_plot = [1, 0]
action_mapping = ['left', 'right', 'collect']
actions_to_color = ['rgb(241, 196, 15)', 'rgb(52, 152, 219)']

fig = go.Figure(layout=go.Layout(
    scene = dict(
        xaxis = dict(
            title='R1 Obs'),
        yaxis = dict(
            title='R2 Obs'),
        zaxis = dict(
            title='Normalized Q'),
    ),

))

for action in actions_to_plot:
    z_2o = normalized_2o_qs[:, action]
    trace_2o = go.Scatter3d(
        x=ot_obs_2o[:, 0], 
        y=ot_obs_2o[:, 1], 
        z=z_2o, 
        name=f"uncertainty {action_mapping[action]}",
        mode='markers',
        marker={
            'size': 2,
            'color': actions_to_color[action],
            'symbol': 'circle'
        }
    )
    fig.add_trace(trace_2o)
        
    z_2g = normalized_2g_qs[:, action]
    trace_2g = go.Scatter3d(
        x=ot_obs_2g[:, 0], 
        y=ot_obs_2g[:, 1], 
        z=z_2g,
        name=f"ground-truth {action_mapping[action]}",
        mode='markers',
        marker={
            'size': 10,
            'color': actions_to_color[action],
            'symbol': 'cross',
            'line': dict(width=0.5, color="black")
        }
    )
    fig.add_trace(trace_2g)
#     mlab.points3d(ot_obs_2g[:, 0], ot_obs_2g[:, 1], z_2g, figure=mfig, color=tuple(actions_to_color[action]), mode='axes', scale_factor=0.055)

#     ax.scatter(ot_obs_2g[:, 0], ot_obs_2g[:, 1], z_2g, cmap='viridis', linewidth=0.5, label=action_mapping[action])

# plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', title='action')
# ax.set_title("Q-values of Lobster environment at node 0 (uncertainty obs)")


fig.show()

In [ ]:
all_zero_2g_qs.max()

In [ ]:
actions_to_plot = [0, 1]
action_mapping = ['left', 'right', 'collect']
actions_to_color = ["orange", "blue"]

fig = plt.figure(figsize=(8, 6), dpi=80)

ax = fig.add_subplot(projection='3d')
ax.view_init(32, 195)

ax.set_xlim(-0.1, 0.9)
ax.set_ylim(-0.1, 0.9)
ax.set_zlim(0.8, 1.4)


ax.set_xlabel("R1 obs")
ax.set_ylabel("R2 obs")
ax.set_zlabel(f"Q")

for action in actions_to_plot:
    z = all_zero_2_qs[:, action]
    ax.scatter(0, 0, z, cmap='viridis', linewidth=0.5, label=action_mapping[action])

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left', title='action')
ax.set_title("Q-values of Lobster environment at node 0 (normal obs)")